# Using AutoSDF

Use this file to generate 3D models using AutoSDF
Make sure the AutoSDF repository is located in in the same parent directory as `completiontools-evaluation`

## Importing and set-up

In [ ]:
# Import packages and Setup everything
import os
import json
import torch
import pytorch3d.io
import sys
import comptools
# some utility function for visualization from AutoSDF
sys.path.insert(0, '../')
import AutoSDF.utils as utils
from AutoSDF.utils.util_3d import init_mesh_renderer, sdf_to_mesh
from AutoSDF.utils.demo_util import get_shape_comp_opt
from AutoSDF.utils.demo_util import get_shape_comp_model
from AutoSDF.utils.qual_util import save_mesh_as_gif, save_meshes_to_file, get_partial_shape_by_voxels, get_partial_shape_by_range, get_shape_comp_input_mesh
from IPython.display import Image as ipy_image
from IPython.display import display

# enable hot reloading
%reload_ext autoreload
%autoreload 2

# set up the results directory
res_dir = 'results'
if not os.path.exists(res_dir): os.mkdir(res_dir)

# Define the options, in this case which GPU we want to use
gpu_id = 2
opt = get_shape_comp_opt(gpu_id=gpu_id)
print(opt.device)

# Setup the object completion model
model = get_shape_comp_model(opt, root = "../AutoSDF")    
model.eval()

# Setup the renderer to display gifs
dist, elev, azim = 1.7, 20, 110
mesh_renderer = init_mesh_renderer(image_size=256, dist=dist, elev=elev, azim=azim, device=opt.device)

## One-click-generation

In [ ]:
sdfPath = "/srvgentjkd98p2/L/Recordings/2015-01 ShapeNetCore/03001627/1a8bbf2994788e2743e99e0cae970928/models/model_normalized_sdf.pt"
jsonPath = '../AutoSDF/demo_data/voxelgrid_half.txt'

# Get the shape completion input
print("Getting the shape completion input")
sdf = torch.load(sdfPath,map_location=torch.device(opt.device))
occ_grid = comptools.read_voxel_file(jsonPath)
shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

# Define the incomplete mesh
print("perform shape completion")
input_mesh = get_shape_comp_input_mesh(shape_comp_input['sdf'], shape_comp_input['sdf_missing'])
input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
gen_mesh = sdf_to_mesh(comp_sdf)          # completed shape

# save as gif
gen_gif_name = f'{res_dir}/shape-comp-struct-input-gen.gif'
save_mesh_as_gif(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name,device = opt.device)
display(ipy_image(gen_gif_name))

# Save to files
print("Saving Meshes")
save_meshes_to_file(gen_mesh, res_dir)

print("Done")

# Step-by-step

## Import the SDF

In [2]:
#sdfPath = "/srvgentjkd98p2/L/Recordings/2015-01 ShapeNetCore/03001627/1a8bbf2994788e2743e99e0cae970928/models/model_normalized_sdf.pt"
sdfPath = "/home/jvermandere/datasets/Paper_examples/Experiment_meshes/cabinet_meshsdf.pt"
sdf = torch.load(sdfPath,map_location=torch.device(opt.device))

In [ ]:
gen_sdf_as_mesh = sdf_to_mesh(sdf) 

# save as gif
gif_name = f'{res_dir}/shape-example.gif'
#save_mesh_as_gif(mesh_renderer, gen_sdf_as_mesh, nrow=1, out_name=gif_name)
ipy_image(gif_name)

## Get The Partial Range

### By Voxels

In [5]:
# Load the JSON file
with open('./demo_data/voxelgrid_half.txt', 'r') as file:
    data = json.load(file)
voxels = data['voxels']
occ_grid = []
for voxel in voxels:
    id = voxel['gridIndex']
    occ_grid.append([id['x'], id['y'], id['z']])

shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

### By Range

In [ ]:
completionPercent = "50"
# range: -1 ~ 1.
# x: left-to-right; y: bottom-to-top; z: front-to-back
min_x, max_x = 0., 1.
min_y, max_y = -1, 1.
min_z, max_z = -1., 1.
input_range = {'x1': min_x, 'x2': max_x, 'y1': min_y, 'y2': max_y, 'z1': min_z, 'z2': max_z}

shape_comp_input = get_partial_shape_by_range(sdf, input_range)

### Get Partial Mesh

In [ ]:
input_mesh = get_shape_comp_input_mesh(shape_comp_input['sdf'], shape_comp_input['sdf_missing'])

# save as gif
gif_name = f'{res_dir}/shape-comp-struct-input-inp.gif'
save_mesh_as_gif(mesh_renderer, input_mesh, nrow=1, out_name=gif_name, device = opt.device)
display(ipy_image(gif_name))

## Shape Completion

In [ ]:
""" perform shape completion """
input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
gen_mesh = sdf_to_mesh(comp_sdf)          # completed shape

# save as gif
gen_gif_name = f'{res_dir}/shape-comp-struct-input-gen.gif'
save_mesh_as_gif(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name,device = opt.device)
display(ipy_image(gen_gif_name))